In [6]:
import numpy as np
import pandas as pd
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from distutils.version import LooseVersion, StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [11]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv
import re
# import cv2 
import os
import glob
import xml.etree.ElementTree as ET

import io
import tensorflow.compat.v1 as tf

from PIL import Image
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile


In [1]:
from object_detection.utils import dataset_util

In [ ]:
columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
data_reformatted = []
data_reformatted

In [ ]:
for k in range(len(data['objects'])):
    for l in range(len(data['objects'][k])):
        line = [data['image'][k]['pathname'],
                data['image'][k]['shape']['r'],
                data['image'][k]['shape']['c'],
                data['objects'][k][l]['category'],
                data['objects'][k][l]['bounding_box']['minimum']['c'],
                data['objects'][k][l]['bounding_box']['minimum']['r'],
                data['objects'][k][l]['bounding_box']['maximum']['c'],
                data['objects'][k][l]['bounding_box']['maximum']['r']]
        data_reformatted.append(line)
pd.DataFrame(data_reformatted,columns=columns)

In [ ]:
new_data = pd.DataFrame(data_reformatted,columns=columns)

In [ ]:
new_data.to_csv('data_reformatted.csv')

In [14]:
#adjusted from: https://github.com/datitran/raccoon_dataset

# converts the csv files for training and testing data to two TFRecords files.
# places the output in the same directory as the input


# from object_detection.utils import dataset_util
# %cd /content/gun_detection/models/

DATA_BASE_PATH = '/home/aga/jupyter/DL-projekt/malaria/'
image_dir = DATA_BASE_PATH +'images/'

def class_text_to_int(row_label):
    if row_label == 'red blood cell':
        return 1
    elif row_label == 'trophozoite':
        return 1
    elif row_label == 'schizont':
        return 1
    elif row_label == 'difficult':
        return 1
    elif row_label == 'ring':
        return 1
    elif row_label == 'leukocyte':
        return 1
    elif row_label == 'gametocyte':
        return 1
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = Image.open(encoded_jpg_io)
        width, height = image.size
        
        filename = group.filename.encode('utf8')
        image_format = b'jpg'
        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes = []
        
        for index, row in group.object.iterrows():
            xmins.append(row['xmin'] / width)
            xmaxs.append(row['xmax'] / width)
            ymins.append(row['ymin'] / height)
            ymaxs.append(row['ymax'] / height)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(class_text_to_int(row['class']))
            
            tf_example = tf.train.Example(features=tf.train.Features(feature={
                'image/height': dataset_util.int64_feature(height),
                'image/width': dataset_util.int64_feature(width),
                'image/filename': dataset_util.bytes_feature(filename),
                'image/source_id': dataset_util.bytes_feature(filename),
                'image/encoded': dataset_util.bytes_feature(encoded_jpg),
                'image/format': dataset_util.bytes_feature(image_format),
                'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
                'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
                'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
                'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
                'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
                'image/object/class/label': dataset_util.int64_list_feature(classes),
            }))
        return tf_example

for csv in ['data_reformatted']:
  writer = tf.io.TFRecordWriter(DATA_BASE_PATH + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(DATA_BASE_PATH + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), DATA_BASE_PATH + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(DATA_BASE_PATH +csv + '.record'))


Successfully created the TFRecords: /home/aga/jupyter/DL-projekt/malaria/data_reformatted.record
